# onepetro_scrape_dev0.2.4

In [38]:
# import lib

import requests
from bs4 import BeautifulSoup
import random
from fake_useragent import UserAgent
import re

import pandas
import numpy

In [18]:
# get all available ip proxies
# store in a list

def get_proxies():
    """
    Function to get list of proxies.
    Can be replaced by any callables as long as it returns [{ip, port.}]
    """
    
    # send request to website
    url = 'https://www.sslproxies.org/'
    headers = {
        'User-Agent' : UserAgent(cache=False).random
    }
    r = requests.get(url=url, headers=headers)
    
    # soup proxy table in response's text
    soup = BeautifulSoup(r.text, 'html.parser')
    t = soup.find(id='proxylisttable')

    # extract ip and port
    proxies = []
    for row in t.tbody.find_all('tr'):
        d = {
            'ip' : row.find_all('td')[0].string,
            'port' : row.find_all('td')[1].string
        }
        proxies.append(d)
        
    return proxies

In [19]:
proxies = get_proxies()

In [20]:
len(proxies)

100

In [21]:
pandas.DataFrame(proxies).head()

,ip,port
0,183.89.14.22,8080
1,157.230.46.178,1080
2,41.67.12.17,8080
3,110.136.205.218,80
4,103.24.107.2,8181


In [52]:
# search query

"""
FOR ADVANCED SEARCH:

https://onepetro.org/search?

q=well+test
+AND+%22machine+learning%22
+AND+-%22simulation%22
+AND+author%3A%22john%22
+AND+affiliation%3A%22spe%22
+AND+publisher%3A%22American+Society+of+Safety+Engineers%22
+AND+journal%3A%22Petroleum+Technology%22
+AND+conference%3A%2211th+ISRM+Congress%22

&peer_reviewed=

&published_between=on

&from_year=1993

&to_year=

&rows=25
"""

def get_N_papers(q):
    """
    Function to get number of papers from search queries.
    Currently only support for simple query (q keyword).
    Advanced queries will be appended in the future dev.
    """

    # send request to onepetro
    url = 'https://onepetro.org/search'
#     headers = {
#         'User-Agent' : UserAgent(cache=False).random
#     }
    # send request to onepetro
    payload = {
        'q'                 : '+'.join(q.split()),
        'peer_reviewed'     : 'off',
        'published_between' : 'off',
        'from_year'         : '',
        'to_year'           : '',
        'rows'              : 1
    }
#     r = requests.get(url=url, params=payload, headers=headers)
    r = requests.get(url=url, params=payload)
    
    # extract number of search results
    soup = BeautifulSoup(r.text, 'html.parser')
    verbose = soup.find('h2').get_text()
    n_str = ''.join(re.findall('\d*[,]\d*', verbose)[0].split(','))
    N = int(n_str)
    
    return N

In [50]:
get_N_papers('well test')

114639

In [53]:
%timeit get_N_papers('gas lift')

1.41 s ± 45.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
# get response from query
# get to know how many results


In [ ]:
# build a function that calls a paper from results
# get all the details of the paper
